<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/166_SubGraphs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


## 🔹 What is a Graph in LangGraph?

* A **graph** is the main workflow definition (`StateGraph`), where you define nodes (functions, models, tools, etc.) and edges (flow between nodes).
* It is your **orchestrator** at the top level.

Example (simplified):

```python
graph = StateGraph(MyState)
graph.add_node("step1", func1)
graph.add_node("step2", func2)
graph.add_edge("step1", "step2")
```

---

## 🔹 What is a Subgraph?

A **subgraph** is a *graph inside a graph*.

* It behaves like a **single node** in the parent graph, but internally it has its own nodes and edges.
* In this example:

  * `insurance_graph` is defined independently (its own nodes: `verify_insurance_check`, `verify_insurance_confirm`).
  * Then it’s added into the main `appointment_graph` as a node:

    ```python
    appointment_graph.add_node("insurance_verification", insurance_graph)
    ```
* This means the parent graph sees only one step `"insurance_verification"`, but inside, there’s a whole verification flow.

---

## 🔹 When to Use Graph vs Subgraph

### ✅ Use a plain graph when:

* Your workflow is **small** (5–10 nodes).
* All steps belong to the same logical layer.
* You don’t expect to reuse the workflow elsewhere.

### ✅ Use subgraphs when:

1. **Modularity / Reuse**

   * E.g., `insurance_graph` could be reused in different workflows (appointments, billing, claims).
   * You build it once, plug it into many parent graphs.

2. **Abstraction / Readability**

   * Keeps the parent graph clean by hiding internal details.
   * Instead of showing 10 insurance-check nodes, the parent just shows `"insurance_verification"`.

3. **Testing in isolation**

   * You can compile and run the subgraph on its own (just like `insurance_graph.invoke(inputs)`).

4. **Team collaboration**

   * Different teams can own different subgraphs and then integrate them into a bigger pipeline.

---

## ⚖️ Analogy

* **Graph** = Whole workflow.
* **Subgraph** = A module or function you import into the workflow.

Think of it like functions in normal Python:

* For small scripts, you can inline logic.
* For bigger systems, you split into reusable functions/classes.

---

✅ So:

* **Graphs** are the building blocks.
* **Subgraphs** are how you keep things modular and reusable.




In [ ]:
from langgraph.graph import StateGraph, END, START
from typing import TypedDict


# Define Shared State
class InsuranceState(TypedDict):
    patient_id: str
    insurance_verified: bool


def verify_insurance_check(state: InsuranceState):
    print("verify_insurance_check")
    if state["patient_id"] is not None:
        return {"insurance_verified": True, "appointment_status": "Insurance verification in progress"}
    else:
        return {"insurance_verified": False, "appointment_status": "Insurance verification pending"}


def verify_insurance_confirm(state: InsuranceState):
    print("verify_insurance_confirm")
    if state["insurance_verified"]:
        return {"appointment_status": "Insurance verified"}
    else:
        return {"appointment_status": "Insurance verification failed"}


# Insurance Verification Subgraph
insurance_graph = StateGraph(InsuranceState)
insurance_graph.add_node("verify_insurance_check", verify_insurance_check)
insurance_graph.add_node("verify_insurance_confirm", verify_insurance_confirm)
insurance_graph.add_edge(START, "verify_insurance_check")
insurance_graph.add_edge("verify_insurance_check", "verify_insurance_confirm")
insurance_graph.add_edge("verify_insurance_confirm", END)
insurance_graph = insurance_graph.compile()


# Define Shared State
class AppointmentState(TypedDict):
    patient_id: str
    appointment_status: str
    insurance_verified: bool
    appointment_scheduled: bool


def schedule_appointment(state: AppointmentState):
    print("schedule_appointment")
    if state["insurance_verified"]:
        return {"appointment_scheduled": True, "appointment_status": "Appointment scheduled"}
    else:
        return {"appointment_scheduled": False, "appointment_status": "Appointment scheduling failed: Insurance issue"}


# Main Appointment Management Graph
appointment_graph = StateGraph(AppointmentState)
# TODO: Add Sub Graph as a node
appointment_graph.add_node("insurance_verification",insurance_graph)
appointment_graph.add_node("schedule_appointment", schedule_appointment)

# Define edges
# TODO: Add the subgraph edge
appointment_graph.add_edge(START,"insurance_verification")
appointment_graph.add_edge("insurance_verification", "schedule_appointment")
appointment_graph.add_edge("schedule_appointment", END)

appointment_graph = appointment_graph.compile()

# Invoke main workflow
inputs = {
    "patient_id": "PT-2025",
}
output = appointment_graph.invoke(inputs)

# TODO: Print the final output
print(output)



For a healthcare provider, **insurance verification is a reusable, domain-specific step** that almost every patient-facing workflow needs:

* Booking an appointment
* Scheduling a surgery
* Handling billing / claims
* Even some telehealth triage flows

If you build it once as a **subgraph**, you can drop it into all of those workflows like a Lego brick.

---

## 🔹 Why a Subgraph is Perfect Here

* **Reusability** → you don’t have to copy-paste the insurance check logic into every agent.
* **Consistency** → every workflow calls the same insurance module, so you don’t risk one agent verifying differently than another.
* **Maintainability** → if the insurance process changes (say new verification APIs), you update the subgraph once, and every agent using it is instantly updated.
* **Abstraction** → the parent graph doesn’t get cluttered with details. At the top level, it just says:

  ```
  intake → insurance_verification → schedule_appointment
  ```

  while inside `insurance_verification` you might have multiple checks and confirmations.

---

## ⚖️ Rule of Thumb

* If a step is **common across many workflows** → make it a **subgraph**.
* If a step is **unique to one workflow** → just keep it as a node in that graph.

---

✅ So yes, in healthcare, things like *insurance verification*, *consent check*, *identity verification*, etc. are **natural candidates for subgraphs**.




💡 you’re thinking about the trade-off between **static subgraphs** and a **dynamic RAG-powered module**. Let’s unpack it:

---

## 🔹 Subgraph Approach

* **Pros**

  * Hard-coded workflow → predictable, deterministic flow.
  * Great for **process steps** (e.g., “call API A, then confirm with API B”).
  * One update propagates everywhere.
* **Cons**

  * If rules change often (e.g., compliance wording, insurer policy), you must **update code** or **retrain/test** the flow.

---

## 🔹 RAG Agent Approach

Imagine your “insurance_verification” isn’t a hardwired flow, but an agent that:

1. Looks up the latest insurer/compliance documents in a vector store.
2. Reads the patient’s insurance info.
3. Generates a decision/recommendation based on the most up-to-date docs.

* **Pros**

  * Automatically stays current with changing policies, as long as your doc store is updated.
  * Lower maintenance — you don’t need to tweak workflow code every time rules shift.
  * Transparent: you can log *which passages* were retrieved for auditing.
* **Cons**

  * Higher variability (model outputs can drift if prompts are loose).
  * More expensive per call (retrieval + LLM reasoning).
  * Compliance teams sometimes want **deterministic workflows**, not probabilistic answers.

---

## 🔹 The Sweet Spot: **Hybrid**

Many orgs do **both**:

* **Subgraph** for the skeleton (e.g., *always do identity check → always do insurance verification → always do scheduling*).
* Inside the **insurance verification node**, you embed a **RAG agent** that consults compliance docs to decide whether to approve, deny, or escalate.

That way:

* ✅ Workflow structure stays **consistent and auditable**.
* ✅ Content / compliance rules stay **dynamic and updateable**.

---

## ⚖️ Rule of Thumb

* If you need **strictly repeatable process steps** → subgraph.
* If you need **up-to-date knowledge and flexible reasoning** → RAG agent.
* If you need **both** → wrap the RAG inside a subgraph node.



🙌 let’s reimagine your  **`insurance_verification`** subgraph with a **RAG-powered agent inside**.

---

## 🔹 Step 1. Set up the RAG

We’ll assume you already have compliance docs indexed in a vector store (Chroma, Pinecone, Weaviate, etc.). Retrieval step pulls the top-k most relevant docs.

```python
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

# Load embeddings + vectorstore
embeddings = OpenAIEmbeddings()
db = Chroma(persist_directory="compliance_docs", embedding_function=embeddings)

retriever = db.as_retriever(search_kwargs={"k": 4})

# Define compliance-aware prompt
compliance_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an insurance compliance agent. "
               "Given the patient insurance info and compliance docs, "
               "decide if coverage is valid and return one of: APPROVED, DENIED, ESCALATE."),
    ("human", "Patient Insurance: {insurance_info}\n\n"
              "Relevant Docs: {context}")
])

# Wrap into chain
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
rag_chain = LLMChain(llm=llm, prompt=compliance_prompt)
```

---

## 🔹 Step 2. Insurance Verification Node

This node now *retrieves docs dynamically* before asking the LLM to decide.

```python
def insurance_verification(state):
    insurance_info = state["insurance_info"]

    # retrieve compliance docs
    docs = retriever.get_relevant_documents(insurance_info)
    context = "\n\n".join([d.page_content for d in docs])

    # run through compliance chain
    decision = rag_chain.run({"insurance_info": insurance_info, "context": context})

    return {"insurance_status": decision.strip()}
```

---

## 🔹 Step 3. Insert into Subgraph

Inside your **patient_appointment** workflow:

```python
from langgraph.graph import StateGraph, END

builder = StateGraph(dict)
builder.add_node("insurance_verification", insurance_verification)

# Example: route based on insurance outcome
def route_insurance(state):
    if state["insurance_status"] == "APPROVED":
        return "schedule"
    else:
        return END

builder.add_conditional_edges(
    "insurance_verification",
    route_insurance,
    {"schedule": "schedule_appointment"}
)
```

---

## 🔹 Key Benefits

* **Reusable subgraph** still exists (drop it into any healthcare workflow).
* **Dynamic compliance logic**: updates as soon as new docs are ingested, no code changes needed.
* **Auditable**: you can log the retrieved docs with each decision for regulators.

---

👉 Do you want me to also add an **audit log step** so that every insurance verification automatically saves:

* Patient insurance info
* Retrieved compliance passages
* Final decision

…so the provider has a compliance trail for every run?


👍 let’s revisit what `add_edge` does in **LangGraph**.

---

## 🔹 `add_edge`

This is how you connect **one node** to **the next** in the workflow.
It tells the graph: “when node A finishes, pass the updated state to node B.”

Example:

```python
builder.add_node("verify", verify_insurance)
builder.add_node("schedule", schedule_appointment)

builder.add_edge("verify", "schedule")
```

This means:

1. The graph will run `verify_insurance`.
2. Whatever state comes out of that node is passed directly into `schedule_appointment`.
3. Execution continues in a straight line.

---

## 🔹 `add_conditional_edges`

Sometimes you don’t always want to go to the same next step. That’s where **conditional edges** come in.

```python
def route_insurance(state):
    if state["insurance_status"] == "APPROVED":
        return "schedule"
    else:
        return "END"

builder.add_conditional_edges(
    "verify",
    route_insurance,
    {"schedule": "schedule"}
)
```

Here:

* If status is **APPROVED** → go to `schedule`.
* Otherwise → stop (`END`).

---

## ⚖️ Rule of Thumb

* Use **`add_edge`** for **fixed, linear flows**.
* Use **`add_conditional_edges`** when the next step depends on the state (branching logic).




## 🔹 RAG Agent

This script builds a **Retrieval-Augmented Generation (RAG) agent** for **current affairs news**. It pulls news from live websites, embeds them, retrieves relevant chunks for a query, and then has an LLM summarize the results into a human-friendly answer.

---

## 🔹 Key Components

### 1. **Document Loading**

It fetches articles from major outlets (BBC, CNN, NYT, Reuters, Al Jazeera) via `WebBaseLoader` and flattens them into a list of documents.

### 2. **Splitting and Embedding**

* Splits text into chunks (`chunk_size=300`, `chunk_overlap=20`) for embedding.
* Stores these chunks in **ChromaDB** with **Ollama embeddings (llama3.2)**.
* Makes a retriever with `.as_retriever()` for queries.

### 3. **First Graph (RAG Retrieval)**

Defines a tiny graph:

* **State:** `RAGGraphState` has `input` and `data`.
* **Node:** `retrieve_data` → runs the retriever against user input and stores results.
* **Edges:** linear flow: `START → retrieve_data → END`.

So `rag_workflow.invoke({"input": question})` returns retrieved documents.

### 4. **Prompt + LLM Chain**

Uses a `ChatPromptTemplate`:

```python
"You are a news analyst summarizing the latest current affairs..."
```

Bound to `ChatOllama(model="llama3.2")` and parsed with `StrOutputParser()`. This ensures the model outputs clean text summaries.

### 5. **Second Graph (Summarization Workflow)**

* **State:** `CurrentAffairsGraphState` with `question`, `retrieved_news`, `generation`.
* **Node:** `generate_current_affairs_summary`:

  * Calls the RAG workflow to get relevant news.
  * Feeds results into the summarization chain.
  * Returns both raw retrieved docs and the generated summary.
* **Edges:** simple linear: `START → generate_current_affairs_summary → END`.

### 6. **Execution**

Runs with:

```python
inputs = {"question": "What are the top global headlines today?"}
response = current_affairs_graph.invoke(inputs)
print(response["generation"])
```

→ Prints a concise news digest from retrieved sources.

---

## 🔹 What to Learn From It

1. **Separation of Concerns:**
   They broke it into two graphs:

   * A retrieval-only graph (modular, reusable).
   * A summarization graph that *calls* the retrieval graph.

2. **Composable Workflows:**
   Notice how `rag_workflow` is invoked inside another node. This shows how **graphs can be nested like functions**, making complex RAG pipelines cleaner.

3. **End-to-End RAG Agent:**
   Classic pattern: **retrieve → inject into prompt → generate answer**.
   This is the same structure you could use for compliance docs, medical knowledge bases, etc.

---

✅ In short: your teacher’s RAG demo is a **current events summarizer**, built from modular graphs — one for retrieval, one for summarization, stitched together.



In [ ]:
from typing import List, TypedDict
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_ollama import OllamaEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langgraph.graph import StateGraph, START, END

# Current Affairs News Sources
news_urls = [
    "https://www.bbc.com/news",
    "https://www.cnn.com/world",
    "https://www.nytimes.com/section/world",
    "https://www.reuters.com/world/",
    "https://www.aljazeera.com/news/"
]

# Load Current Affairs Documents
docs = [WebBaseLoader(url).load() for url in news_urls]
docs_list = [item for sublist in docs for item in sublist]

# Split the articles for embeddings
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=300, chunk_overlap=20
)
doc_splits = text_splitter.split_documents(docs_list)

# Store and Retrieve Current Affairs with ChromaDB
vectorstore = Chroma.from_documents(
    documents=doc_splits,
    collection_name="current-affairs-news",
    embedding=OllamaEmbeddings(model="llama3.2"),
)
retriever = vectorstore.as_retriever()


class RAGGraphState(TypedDict):
    input: str
    data: str


# TODO: Use the retriever and retrieve the matching news
def retrieve_data(state: RAGGraphState):
    print("---Retrieve Data")
    input = state["input"]
    data = retriever.invoke(input)
    return {"data": data}


def create_rag_workflow():
    workflow = StateGraph(RAGGraphState)
    workflow.add_node("retrieve_data", retrieve_data)
    workflow.add_edge(START, "retrieve_data")
    workflow.add_edge("retrieve_data", END)
    return workflow.compile()


rag_workflow = create_rag_workflow()


# Prompt for Current Affairs News Summarization
prompt = ChatPromptTemplate.from_template(
    """
    You are a news analyst summarizing the latest current affairs.
    Use the retrieved articles to provide a concise summary.
    Highlight key global events and developments.

    Question: {question}
    News Articles: {context}
    Summary:
    """
)
model = ChatOllama(model="llama3.2")
current_affairs_chain = (
        prompt | model | StrOutputParser()
)


class CurrentAffairsGraphState(TypedDict):
    question: str
    retrieved_news: List[str]
    generation: str


# TODO: Summarize the news
# News Summary Generation Node
def generate_current_affairs_summary(state):
    print("---GENERATE CURRENT AFFAIRS SUMMARY---")
    question = state["question"]
    # TODO: Invoke the rag workflow
    retrieved_news = rag_workflow.invoke({"input": question})
    generation = current_affairs_chain.invoke({"question": question,"context": retrieved_news["data"]})
    return {"question": question, "retrieved_news": retrieved_news,"generation": generation}


# Current Affairs News Workflow Definition
def create_current_affairs_workflow():
    workflow = StateGraph(CurrentAffairsGraphState)
    workflow.add_node("generate_current_affairs_summary", generate_current_affairs_summary)
    workflow.add_edge(START, "generate_current_affairs_summary")
    workflow.add_edge("generate_current_affairs_summary", END)
    return workflow.compile()


# Execute the Current Affairs News Workflow
current_affairs_graph = create_current_affairs_workflow()

inputs = {"question": "What are the top global headlines today?"}

response = current_affairs_graph.invoke(inputs)

print("\n--- CURRENT AFFAIRS SUMMARY ---")
print(response["generation"])
